In [1]:
'''Functions'''

import numpy as np
import pandas as pd
#import scipy as sci
#import sympy as sym
#import statistics as stat
import random as rng
#import itertools as ite
#import time
#import math
import plothist as ph
#import vector as vec
#import awkward as ak
#------------------
import matplotlib.pyplot as plt
#import matplotlib.animation as manim
import matplotlib_inline
#from matplotlib.ticker import AutoMinorLocator
#import cartopy.crs as ccrs
#from cartopy.io.img_tiles import GoogleTiles
#import timeit
#from IPython.display import display
from mpl_toolkits import mplot3d
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.ticker import AutoMinorLocator 
from matplotlib.ticker import MaxNLocator

#---------
#import json
import os
#import gc
#import sys
#----------
#import awkward as ak
#import uproot
#import vector

def lorentz_boost(four_vec, beta): #lorents boost for any frame
    beta = np.asarray(beta).reshape(3,)
    beta_sqr = np.dot(beta, beta)
    if beta_sqr == 0.0:
        return four_vec
    gamma = (1.0 - beta_sqr)**(-0.5)
    gammafrac = (gamma**2 / (1+gamma))
    bx, by, bz = beta

    loren = np.array([
        [gamma, -gamma*bx, -gamma*by, -gamma*bz],
        [-gamma*bx, 1+gammafrac*bx*bx, gammafrac*bx*by, gammafrac*bx*bz],
        [-gamma*by, gammafrac*by*bx, 1+gammafrac*by*by, gammafrac*by*bz],
        [-gamma*bz, gammafrac*bz*bx, gammafrac*bz*by, 1+gammafrac*bz*bz]
    ])
    return np.matmul(loren,four_vec)

def isotropic_direction(): #Return a random uniform selection of angles for spherical coordinates
    cos_theta = np.random.uniform(-1, 1)
    sin_theta = np.sqrt(1 - cos_theta**2)
    phi = np.random.uniform(0, 2 * np.pi)
    return np.array([sin_theta * np.cos(phi), sin_theta * np.sin(phi), cos_theta])

def four_vect(E, p3): #Given E and 3vec of momentum, make 4vec. 
    return np.array([E, *p3])

def sph_crt(phi, theta): #To cartesian from polar
    z = np.cos(theta)
    cos_phi = np.cos(phi)
    sin_phi = np.sin(phi)
    sin_theta = np.sin(theta)
    x = sin_theta * cos_phi
    y = sin_theta * sin_phi
    return x,y,z

def crt_sph(x,y,z): #to polar from cartesian
    theta = np.arccos(z)
    phi = np.arctan(y/x)
    return phi, theta

def calc_p4(E, p, costh, phi):  #Calculate the four vector of a particle (E,px,py,pz), Z axis is beam axis, phi is azimuthal
    sinth = np.sqrt(1 - costh**2)
    px = p * sinth * np.cos(phi)
    py = p * sinth * np.sin(phi)
    pz = p * costh
    return np.array([E, px, py, pz])

def calc_rapid(E, p, costh): #Rapidity
    return 0.5*np.ln( (E + p*costh) / (E - p*costh) )

def calc_prapid(costh): #pseudorapidity, usually eta
    return np.arctanh(costh)

def calc_betagamma(E,p): #Returns a tuple of beta v/c, and gamma. C = 1. 
    return p/E, E / np.sqrt(E**2-p**2)

def calc_pt(p, costh, phi): #Calculate transverse momentum
    sinth = np.sqrt(1 - costh**2)
    return ((p * sinth * np.cos(phi))**2 + (p * sinth * np.sin(phi))**2)**0.5

def calc_pl(p,costh): #calculate along beam axis (longitude) momentum
    return p * costh

def calc_InvM2(E,p):
    return E**2-p**2

def calc_flight_dist(dr,dz):
    return np.sqrt(dr**2+dz**2)

def calc_eff(passed_mask):
    return np.sum(passed_mask) / len(passed_mask)

def is_true_KS(ks_mcPDG, pi1_mother_PDG, pi2_mother_PDG):
    """Check if candidate is true K_S^0 based on MC truth."""
    return (ks_mcPDG == 310) & (pi1_mother_PDG == 310) & (pi2_mother_PDG == 310)


def plot_sim(datas, particle, quantity, src_names, bin_c=100, ax=None, src_count=3, scale_weight=1.0, bin_edges=None, err=False): 
    '''
    Args
    -> datas, list of data
    -> particle, name as a string, ie (D0SL)
    -> quantity, as a string, ie (InvM)
    -> d_id, filtering id, integer
    -> src_names, list of decay source names (in order of data list)
    -> bin_c, bin number
    -> ax, axis of subplot none by default for a single plot. axs[i] object otherwise
    -> src_count, count of sources to plot, 3 default
    -> scale_weight, normalization for all MC samples combined for decay ID and every entry
    -> bin_edges, for custom fitting
    -> err, for including attempt at source uncertainty
    -> filtarg, sorting filter or column (please use only ones that take whole numbers), decayModeID by default
    Future iterations will reduce parameter needs, improve the filtering and collecting algorithm, adjust layouts, and add more functionality
    '''
    phdl = particle+'_'+quantity
    
    data_list = []
    weight_list = []
    label_list = []
    
    for i in range(src_count):
        try:
            data = datas[i].query(f'{particle}_{filtarg}=={d_id}')[phdl] #Filter to quantities on filtering arg
        except:
            data = datas[i].query(f'{filtarg}=={d_id}')[phdl] #Yes, this is the laziest fix in the world for ncandidates

        data_list.append(data) #Grab all filtered ID data of interest via the loop
        weight_list.append(np.full(len(data),scale_weight)) #To prevent issues with non uniformity
        label_list.append(f'{filtarg}: {d_id}\nSource: {src_names[i]}') #Grab labels
    
    if ax is not None and not err:
        ax.hist(data_list, bins=bin_edges, stacked=True,weights=weight_list,label=label_list,edgecolor='black',align='mid')  
    elif ax is not None:
        ax.hist(data_list, bins=bin_edges, stacked=True,weights=weight_list,label=label_list,edgecolor='black')
        counts, edges, __ = ax.hist(data_list, bins=bin_edges, stacked=False, weights=weight_list, alpha=0)
        top_y_values = np.sum(counts, axis=0)
        centers = 0.5 * (edges[:-1] + edges[1:])
        step_size = bin_edges[1] - bin_edges[0]
        errors = np.sqrt(top_y_values)
        ax.bar(centers,height=2*errors,bottom=top_y_values - errors,alpha=0.8, #This was a pain to get working, have to draw invisible histogram to avoid the stacking
            color='none',edgecolor='black',hatch='////',width=step_size,label='Sim Uncertainty')
    else:
        plt.hist(data_list, bins=bin_edges,stacked=True,weights=weight_list,label=label_list,edgecolor='black')    